In [ ]:
!pip install --upgrade --force-reinstall transformers==4.39.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 712.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import pandas as pd
import kagglehub
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer

In [ ]:
# Подгружаем данные
path_data = kagglehub.dataset_download("jerryqu/reddit-conversations")

print("Path to data:", path_data)

100%|██████████| 1.83M/1.83M [00:00<00:00, 112MB/s]

Extracting files...
Path to data: /root/.cache/kagglehub/datasets/jerryqu/reddit-conversations/versions/1


In [ ]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/jerryqu/reddit-conversations/versions/1/casual_data_windows.csv', index_col = 0)
df

,0,1,2
0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."
2,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und...","cool. I've been thinking of getting one, my ph..."
3,What kind of phone(s) do you guys have?,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
4,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?,I love it. I can't think of anything I don't l...
...,...,...,...
56292,I just won a $570 PlayStation 4 Pro at Timezone!,Are you from Singapore?Because I recognised th...,It's also in the Philippines! But our prizes a...
56293,Are you from Singapore?Because I recognised th...,It's also in the Philippines! But our prizes a...,Also in Australia!
56294,It's also in the Philippines! But our prizes a...,Also in Australia!,and New Zealand!
56295,Also in Australia!,and New Zealand!,And my axe!


In [ ]:
# Function to convert float to string
def float_to_string(value):
    if isinstance(value, float):
        return str(value)
    return value

df = df.applymap(float_to_string)

<ipython-input-4-a3c8f2c13b38>:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(float_to_string)


In [ ]:
catalog = df.iloc[:, :3].values.flatten().tolist()

dataset = []
for i in range(1, len(catalog), 2):
    q = catalog[i-1]
    a = catalog[i]
    dataset.append([q,a])

In [ ]:
import string
import re
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

def cleaning_text(text):
    text_nopunct = ''.join([char for char in text if char not in string.punctuation])
    text_tokenise = re.split('\W+', text_nopunct)
    text_nostopword = [word for word in text_tokenise if word not in stopword]
    text_stemming = [ps.stem(word) for word in text_nostopword]
    text_lemmatizing = [wn.lemmatize(word) for word in text_stemming]
    text_cleaned = [item for item in text_lemmatizing if item != '']
    return text_cleaned

catalog_cleaned = [cleaning_text(char) for char in dataset]
dataset = [' '.join(char) for char in catalog_cleaned if ' '.join(char).strip()]

In [ ]:
# Split data
def split_data(data, train_ratio = 0.7, val_ratio = 0.15, test_ratio = 0.15):
    train_data, temp_data = train_test_split(data, test_size = (1-train_ratio), random_state = 42)
    val_data, test_data = train_test_split(temp_data, test_size= test_ratio/(test_ratio + val_ratio), random_state = 42)
    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(dataset)

In [ ]:
# Add padding token to the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Tokenize data
train_encodings = tokenizer(train_data, padding = True, truncation = True, max_length = 50, return_tensors = 'pt')
val_encodings = tokenizer(val_data, padding = True, truncation = True, max_length = 50, return_tensors = 'pt')
test_encodings = tokenizer(test_data, padding = True, truncation = True, max_length = 50, return_tensors = 'pt')

In [ ]:
# Print shapes of input_ids
print(f"Train data shape: {len(train_encodings['input_ids'])}")
print(f"Validation data shape: {len(val_encodings['input_ids'])}")
print(f"Test data shape: {len(test_encodings['input_ids'])}")

Train data shape: 59071
Validation data shape: 12658
Test data shape: 12659


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import GPT2LMHeadModel, AdamW, GPT2Tokenizer
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
import math

In [ ]:
def train_model(model, optimizer, scheduler, num_epochs, train_loader, val_loader, device):
    model.to(device)
    model.train()

    train_losses = []
    val_losses = []
    val_ppl = []

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            inputs = batch[0].to(device)
            masks = batch[1].to(device)

            outputs = model(inputs, attention_mask=masks, labels=inputs)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            if scheduler is not None:
                scheduler.step()
            optimizer.zero_grad()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        total_val_loss = 0
        for batch in val_loader:
            inputs = batch[0].to(device)
            masks = batch[1].to(device)

            with torch.no_grad():
                outputs = model(inputs, attention_mask=masks, labels=inputs)
                loss = outputs.loss
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_ppl.append(math.exp(avg_val_loss))

    return train_losses, val_losses, val_ppl

In [ ]:
# Load pre-trained model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Hyperparameters
batch_size = 8
num_epoch = 8
learning_rate = 5e-5
max_lenght = 50

# Prepare datasets and dataloaders
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'])
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'])
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'])

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

optimizer = AdamW(model.parameters(), lr = learning_rate)

total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 100, num_training_steps = total_steps)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_losses, val_losses, val_ppl = train_model(model, optimizer, scheduler, num_epoch, train_loader, val_loader, device)

print('Training Results:')
for epoch in range(num_epoch):
    print(f'Epoch {epoch + 1}/{num_epoch}')
    print(f'Train loss: {train_losses[epoch]:.4f}')
    print(f'Validation loss: {val_losses[epoch]:.4f}')
    print(f'Validation Perplexity: {val_ppl[epoch]:.4f}')
    print()
print('Training Completed.')

Training Results:
Epoch1/3
Train loss: 1.4276
Validation loss: 1.2839
Validation Perplexity: 3.6109

Epoch2/3
Train loss: 1.0934
Validation loss: 1.1243
Validation Perplexity: 3.0780

Epoch3/3
Train loss: 0.7662
Validation loss: 1.0948
Validation Perplexity: 2.9886

Training Completed.


In [ ]:
torch.save(model, 'model_gpt.pth')

In [ ]:
torch.save(model.state_dict(), 'saved_weights.pt')

In [ ]:
#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
!pip install rouge-score

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
# Функция для расчета BLEU
def calculate_bleu_score(references, candidates):
    scores = []
    for ref, cand in zip(references, candidates):
        ref_tokens = [ref.split()]  # Референсный текст как список токенов
        cand_tokens = cand.split()  # Кандидат как список токенов
        score = sentence_bleu(ref_tokens, cand_tokens)  # Расчет BLEU
        scores.append(score)
    return sum(scores) / len(scores)  # Средний BLEU

# Функция для расчета ROUGE
def calculate_rouge_score(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {"rouge1": [], "rouge2": [], "rougeL": []}
    for ref, cand in zip(references, candidates):
        score = scorer.score(ref, cand)  # Расчет ROUGE
        scores["rouge1"].append(score['rouge1'].fmeasure)
        scores["rouge2"].append(score['rouge2'].fmeasure)
        scores["rougeL"].append(score['rougeL'].fmeasure)
    return {key: sum(value) / len(value) for key, value in scores.items()}  # Средний ROUGE

# Функция для генерации текста
def generate_text(model, tokenizer, prompts, max_length=50):
    model.eval()
    candidates = []
    for prompt in prompts:
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
        output = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
        candidate = tokenizer.decode(output[0], skip_special_tokens=True)
        candidates.append(candidate)
    return candidates


In [ ]:
# Примеры промптов и референсов
prompts = [
    "What kind of phone",
    "What your favourite chocolate",
    "What your favorite to draw",
]

references = [
    "What kind of phone(s) do you guys have?",
    "What's your favourite chocolate/candy bar?",
    "What's your favorite thing to draw?",
]

# Генерация текста
candidates = generate_text(model, tokenizer, prompts)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# Расчет метрик
bleu_score = calculate_bleu_score(references, candidates)
rouge_scores = calculate_rouge_score(references, candidates)

# Вывод результатов
print("BLEU:", bleu_score)
print("ROUGE:", rouge_scores)

# Отображение сгенерированных текстов
for i, (prompt, candidate, reference) in enumerate(zip(prompts, candidates, references)):
    print(f"Пример {i + 1}:")
    print(f"Prompt: {prompt}")
    print(f"Generated: {candidate}")
    print(f"Reference: {reference}")
    print()

BLEU: 1.4882224320109662e-78
ROUGE: {'rouge1': 0.6568627450980392, 'rouge2': 0.3333333333333333, 'rougeL': 0.6568627450980392}
Пример 1:
Prompt: What kind of phone
Generated: What kind of phone what unusu thing phone
Reference: What kind of phone(s) do you guys have?

Пример 2:
Prompt: What your favourite chocolate
Generated: What your favourite chocolateweet chocol
Reference: What's your favourite chocolate/candy bar?

Пример 3:
Prompt: What your favorite to draw
Generated: What your favorite to draw
Reference: What's your favorite thing to draw?



/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
def generate_response(model, tokenizer, input_text, max_length=50, top_k=50, top_p=0.95):
    model.eval()
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # Генерация ответа
    output = model.generate(
        input_ids,
        max_length=max_length,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Пример использования чат-бота
def chat_with_bot():
    print("Чат-бот запущен. Введите 'exit' для выхода.")
    while True:
        user_input = input("Вы: ")
        if user_input.lower() == 'exit':
            break

        response = generate_response(model, tokenizer, user_input)
        print(f"Бот: {response}")

# Запуск чат-бота
chat_with_bot()

Чат-бот запущен. Введите 'exit' для выхода.
Вы: what phone to use
Бот: what phone to use phone tell maroon nine dt
Вы: where are you
Бот: where are you your
Вы: i like chocolate
Бот: i like chocolateymeatsdo mean chocol
Вы: exit
